In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
!pip install tool

In [ ]:
!pip install tensorflow numpy matplotlib

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data_preprocessed_python
os.getcwd()
os.chdir('/content/drive/My Drive')

In [ ]:
# Define image preprocessing function
def preprocess_image(image_path, target_size=(1000, 600)):
    img = Image.open(image_path)
    img = img.resize(target_size)  # Resize image to target size
    img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
    return img_array

In [ ]:
# Load paths for 10 image files and 1 test image
image_folder_path = '/content/drive/My Drive/spectrograms'  # Path to your dataset folder containing images

# Use os.listdir to get all files in the folder
image_files = [f for f in os.listdir(image_folder_path) if f.endswith('.png')]
# Filter for image files (assuming they are PNGs)

# Create image paths based on found files
image_paths = [os.path.join(image_folder_path, f) for f in image_files[:10]]
# Limit to 10 images for training

test_image_path = '/content/drive/My Drive/spectrograms/test_audio.png'  # Path to your test image

# Preprocess images
X_train = np.array([preprocess_image(img_path) for img_path in image_paths])
y_train = np.array([i for i in range(len(image_paths))])  # Labels for each song (adjusted to the number of images)

# Preprocess test image
test_image_data = preprocess_image(test_image_path)

In [ ]:
# Build CNN model for classifying the spectrogram images
def build_model(input(shape)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')  # Assuming we have 10 classes (songs)
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Prepare training data for CNN
X_train_normalized = X_train / 255.0  # Normalize pixel values

# Convert to RGB (dropping the alpha channel)
X_train_normalized = X_train_normalized[:, :, :, :3]  # Select only the first 3 channels (RGB)
X_train_normalized = X_train_normalized.reshape(-1, 600, 1000, 3)  # Reshape with correct dimensions

# Build model
model = build_model((1000, 600, 3))

# Train model
model.fit(X_train_normalized, y_train, batch_size=2, epochs=10)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 59s 12s/step - accuracy: 0.0000e+00 - loss: 3.0835
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 78s 10s/step - accuracy: 0.0000e+00 - loss: 2.3228
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 79s 10s/step - accuracy: 0.0000e+00 - loss: 2.2952
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 10s/step - accuracy: 0.0000e+00 - loss: 2.3034
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 81s 10s/step - accuracy: 0.0000e+00 - loss: 2.3028
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 85s 11s/step - accuracy: 0.0000e+00 - loss: 2.3030
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 80s 10s/step - accuracy: 0.0000e+00 - loss: 2.3029
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 86s 11s/step - accuracy: 0.0000e+00 - loss: 2.3029
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 81s 10s/step - accuracy: 0.2069 - loss: 2.3031
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 79s 10s/step - accuracy: 0.0819 - loss: 2.3026


In [ ]:
# Predict similarity with the test image
test_image_data_reshaped = np.expand_dims(test_image_data, axis=0) # Add batch dimension

# Convert the test image to RGB by dropping the alpha channel
test_image_data_reshaped = test_image_data_reshaped[:, :, :, :3]

predictions = model.predict(test_image_data_reshaped)
most_similar_photo_index = np.argmax(predictions)

print(f"The test image is most similar to song_{most_similar_photo_index + 1}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 543ms/step
The test image is most similar to song_7


In [ ]:
# Import the classification_report function
from sklearn.metrics import classification_report

# Evaluate the model using accuracy metrics (optional)
y_pred_classes = np.argmax(model.predict(X_train_normalized), axis=1)
print(classification_report(y_train, y_pred_classes))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.10      1.00      0.18         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1

    accuracy                           0.10        10
   macro avg       0.01      0.10      0.02        10
weighted avg       0.01      0.10      0.02        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
